In [29]:
import PyPDF2
import os
from PIL import Image
import pandas as pd
import tabula as tb
import os
import pandas as pd
import PIL
import mysql.connector as sql

In [36]:
# Extracting Data Table From PDF file
# Declare Variables
dirr = "C:/Users/afkaa/Google Drive/Multimatics/Data/UINSA/"
table_dir = "table_output"
list_pdf = []
list_table = []

# Create Folder if not exist
if os.path.isdir(table_dir) == False:
    os.mkdir(table_dir) 

# Find pdf file in designated Directory
for file in os.listdir(dirr):
    if file.startswith("Daftar Hadir UINSA") and file.endswith(".pdf"):
        print("Found File : " + file)
        list_pdf.append(os.path.join(dirr, file))

# Extract Table Data from listed Pdf File
for pdf in list_pdf:
    print("Extracting Table Data From", pdf)
    dfs = tb.read_pdf(pdf, pages='all')
    #Convert to Pandas
    df = pd.concat(dfs)
    df = df.drop(columns=['Added Time', 'Referrer Name', 'Task Owner'])
    list_table.append(df)

    
### Create Separate File or Combine the whole table
## Separate File
num_table = 0
for table in list_table:
    table.to_csv(os.path.join(table_dir,"table-" + str(num_table) + ".csv"), index=False)
    print("Created CSV file Named : " + "table-" + str(num_table) + ".csv")
    num_table += 1

## Combine Table
#df_full = pd.concat(list_table)

#Save to CSV
#df_full.to_csv("combine.csv",index=False)
#print(df_full.dtypes)

Found File : Daftar Hadir UINSA 2021_06 - 11 Nov 2021.pdf
Found File : Daftar Hadir UINSA 2021_12 - 17 Okt 2021.pdf
Found File : Daftar Hadir UINSA 2021_2 - 7 Okt 2021.pdf
Found File : Daftar Hadir UINSA 2021_27 Sep - 2 Okt 2021.pdf
Found File : Daftar Hadir UINSA 2021_7 - 12 Okt 2021.pdf
Extracting Table Data From C:/Users/afkaa/Google Drive/Multimatics/Data/UINSA/Daftar Hadir UINSA 2021_06 - 11 Nov 2021.pdf
Extracting Table Data From C:/Users/afkaa/Google Drive/Multimatics/Data/UINSA/Daftar Hadir UINSA 2021_12 - 17 Okt 2021.pdf
Extracting Table Data From C:/Users/afkaa/Google Drive/Multimatics/Data/UINSA/Daftar Hadir UINSA 2021_2 - 7 Okt 2021.pdf
Extracting Table Data From C:/Users/afkaa/Google Drive/Multimatics/Data/UINSA/Daftar Hadir UINSA 2021_27 Sep - 2 Okt 2021.pdf
Extracting Table Data From C:/Users/afkaa/Google Drive/Multimatics/Data/UINSA/Daftar Hadir UINSA 2021_7 - 12 Okt 2021.pdf
Created CSV file Named : table-0.csv
Created CSV file Named : table-1.csv
Created CSV file Name

In [53]:
df_full[5950:5960]

,NIM,Nama Lengkap,Program Studi,Kelas Pelatihan,Pelatihan Hari Ke-,Tanda Tangan Jelas
16,D92217075,SELMA MAYLA SUNNI,Pendidikan Bahasa Arab,Virtual Classroom 11 | 27Sep - 2 Oct 2021,5.0,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN
19,I73217060,JIEHAD KAVALERA,Sosiologi,Virtual Classroom 4 | 27 Sep- 2 Oct 2021,5.0,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN
22,H74217025,Desinta Fadeani S,Ilmu Kelautan,Virtual Classroom 10 | 27Sep - 2 Oct 2021,5.0,NaN
23,E93217082,Nihayatul Ulum,Ilmu Al Qur`an dan Tafsir,Virtual Classroom 9 | 27 Sep- 2 Oct 2021,5.0,NaN
0,C92217161,Nur Hikmah Fajriani,Hukum Ekonomi Syariah\r(Muamalah),Virtual Classroom 8 | 27 Sep\r- 2 Oct 2021,5.0,NaN
1,D99217073,Neviantikna Inggia\rCahyaning Putri,Pendidikan Islam Anak Usia\rDini,Virtual Classroom 12 | 27\rSep - 2 Oct 2021,5.0,NaN


In [38]:
### Declare Variable
img_out = "img_out"
table_image_dir = "tabel_image_output"
csv_input = []
row_input = 0
file_num = 0

### Indexing Table File
for file in os.listdir(os.path.join(os.getcwd(), table_dir)):
    if file.startswith("table") and file.endswith(".csv"):
        print("Found CSV file for input: " + file)
        csv_input.append(file)

### Check if Table Image Output Directory Created and Create it if False
if os.path.isdir(table_image_dir) == False:
    os.mkdir(table_image_dir)
        
### Check if Image Output Directory Created and Create it if False
if os.path.isdir(img_out) == False:
    os.mkdir(img_out)        
        
### Extracting Image from PDF by CSV list
for index in range(len(csv_input)):
    # Read and Convert Column to Str
    df = pd.read_csv(csv_input[index])
    df = df.astype({"Tanda Tangan Jelas": str})
    
    pdf_input = PyPDF2.PdfFileReader(open(list_pdf[index], "rb"))
    
    print("Processing File " + csv_input[index] + " and " + list_pdf[index])
    
    n_pages = pdf_input.getNumPages()
    for page in range(n_pages):
        pages = pdf_input.getPage(page)
        xObject = pages['/Resources']['/XObject'].getObject()

        for obj in xObject:
            if xObject[obj]['/Subtype'] == '/Image':
                size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
                data = xObject[obj].getData()
                if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                    mode = "RGB"
                else:
                    mode = "P"

                if xObject[obj]['/Filter'] == '/FlateDecode':
                    img = Image.frombytes(mode, size, data)
                    saving_name = img_out + '/' + "pdf-" + str(file_num) + "-" + obj[1:]  + ".png"
                    img.save(saving_name)
                    df.at[row_input, "Tanda Tangan Jelas"] = saving_name
                elif xObject[obj]['/Filter'] == '/DCTDecode':
                    img = open(img_out + '/' + "pdf-" + str(file_num) + "-" +  obj[1:] + ".jpg", "wb")
                    img.write(data)
                    img.close()
                elif xObject[obj]['/Filter'] == '/JPXDecode':
                    img = open(img_out + '/' + "pdf-" + str(file_num) + "-" +  obj[1:] + ".jp2", "wb")
                    img.write(data)
                    img.close()
            
                row_input += 1
    
    df.to_csv(os.path.join(table_image_dir,"hasil-" + str(file_num) + ".csv"),index=False)
    print("Saving CSV Index Image to : " + "hasil-" + str(file_num) + ".csv")
    row_input = 0
    file_num += 1

Found CSV file for input: table-0.csv
Found CSV file for input: table-1.csv
Found CSV file for input: table-2.csv
Found CSV file for input: table-3.csv
Found CSV file for input: table-4.csv
Processing File table-0.csv and C:/Users/afkaa/Google Drive/Multimatics/Data/UINSA/Daftar Hadir UINSA 2021_06 - 11 Nov 2021.pdf
Saving CSV Index Image to : hasil-0.csv
Processing File table-1.csv and C:/Users/afkaa/Google Drive/Multimatics/Data/UINSA/Daftar Hadir UINSA 2021_12 - 17 Okt 2021.pdf
Saving CSV Index Image to : hasil-1.csv
Processing File table-2.csv and C:/Users/afkaa/Google Drive/Multimatics/Data/UINSA/Daftar Hadir UINSA 2021_2 - 7 Okt 2021.pdf
Saving CSV Index Image to : hasil-2.csv
Processing File table-3.csv and C:/Users/afkaa/Google Drive/Multimatics/Data/UINSA/Daftar Hadir UINSA 2021_27 Sep - 2 Okt 2021.pdf
Saving CSV Index Image to : hasil-3.csv
Processing File table-4.csv and C:/Users/afkaa/Google Drive/Multimatics/Data/UINSA/Daftar Hadir UINSA 2021_7 - 12 Okt 2021.pdf
Saving CSV

In [ ]:
# Read and Convert Column to Str
img_out = "img_out"
csv_input = "test.csv"
row_input = 0
file_num = 0

df = pd.read_csv(csv_input)
df = df.astype({"Tanda Tangan Jelas": str})
print(df.dtypes)

# Check if Output Directory Created and Create it if False
if os.path.isdir(img_out) == False:
    os.mkdir(img_out)

for file in list_file:
    pdf_input = PyPDF2.PdfFileReader(open(file, "rb"))
    
    n_pages = pdf_input.getNumPages()
    for page in range(n_pages):
        pages = pdf_input.getPage(page)
        xObject = pages['/Resources']['/XObject'].getObject()

        for obj in xObject:
            if xObject[obj]['/Subtype'] == '/Image':
                size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
                data = xObject[obj].getData()
                if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                    mode = "RGB"
                else:
                    mode = "P"

                if xObject[obj]['/Filter'] == '/FlateDecode':
                    img = Image.frombytes(mode, size, data)
                    saving_name = img_out + '/' + "pdf-" + str(file_num) + "-" + obj[1:]  + ".png"
                    img.save(saving_name)
                    df.at[row_input, "Tanda Tangan Jelas"] = saving_name
                elif xObject[obj]['/Filter'] == '/DCTDecode':
                    img = open(img_out + '/' + "pdf-" + str(file_num) + "-" +  obj[1:] + ".jpg", "wb")
                    img.write(data)
                    img.close()
                elif xObject[obj]['/Filter'] == '/JPXDecode':
                    img = open(img_out + '/' + "pdf-" + str(file_num) + "-" +  obj[1:] + ".jp2", "wb")
                    img.write(data)
                    img.close()
            
                row_input += 1
    file_num += 1
    
df.to_csv("hasil.csv",index=False)

In [ ]:
import PyPDF2
import os
from PIL import Image
import pandas as pd

# Variable Declaration
pdf_input = PyPDF2.PdfFileReader(open("C:/Users/afkaa/Google Drive/Multimatics/Data/Daftar Hadir UINSA 2021_06 - 11 Nov 2021.pdf", "rb"))
img_out = "img_out"
csv_input = "test.csv"
row_input = 0

# Read and Convert Column to Str
df = pd.read_csv(csv_input)
df = df.astype({"Tanda Tangan Jelas": str})
print(df.dtypes)

# Check if Output Directory Created and Create it if False
if os.path.isdir(img_out) == False:
    os.mkdir(img_out)
    
n_pages = pdf_input.getNumPages()
for page in range(n_pages):
    pages = pdf_input.getPage(page)
    xObject = pages['/Resources']['/XObject'].getObject()

    for obj in xObject:
        if xObject[obj]['/Subtype'] == '/Image':
            size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
            data = xObject[obj].getData()
            if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                mode = "RGB"
            else:
                mode = "P"

            if xObject[obj]['/Filter'] == '/FlateDecode':
                img = Image.frombytes(mode, size, data)
                saving_name = img_out + '/' + obj[1:] + ".png"
                img.save(saving_name)
                df.at[row_input, "Tanda Tangan Jelas"] = saving_name
            elif xObject[obj]['/Filter'] == '/DCTDecode':
                img = open(img_out + '/' +  obj[1:] + ".jpg", "wb")
                img.write(data)
                img.close()
            elif xObject[obj]['/Filter'] == '/JPXDecode':
                img = open(img_out + '/' +  obj[1:] + ".jp2", "wb")
                img.write(data)
                img.close()
            
            row_input += 1

df.to_csv("hasil.csv",index=False)

In [35]:
# Percobaan Import Ke MySQL

def convertToBinaryData(filename):
    # Convert digital data to binary format
    try:
        with open(filename, 'rb') as file:
            binaryData = file.read()
        return binaryData
    except:
        return "zero"

def insertToMySQL(dataOneRow):
    try:
        connection = sql.connect(host='localhost', 
                                database='mahasiswa', 
                                user='lanang_afkaar', 
                            password='1q2w3e4r5t')
        cursor = connection.cursor()
        sql_insert = """INSERT INTO UINSA
                        (NIM, NamaLengkap, ProgramStudi, 
                        KelasPelatihan, PelatihanHariKe, TandaTanganJelas)
                        VALUES (%s, %s, %s, %s, %s, %s)"""
        result  = cursor.execute(sql_insert, dataOneRow)
        connection.commit()
        return 1
    
    except sql.Error as error:
        print("Failed inserting BLOB data into MySQL table {}".format(error))
        return 0

# Convert data into tuple format
list_file = []
count = 0

for file in os.listdir(os.getcwd()):
    if file.startswith("hasil") and file.endswith(".csv"):
        print("Found File processing :" + file)
        list_file.append(file)
        
for file in list_file:
    df_mahasiswa = pd.read_csv(file)
    print("Processing Upload for :" + file)
    for index, row in df_mahasiswa.iterrows():
        NIM = row['NIM']
        NamaLengkap = row['Nama Lengkap']
        ProgramStudi = row['Program Studi']
        KelasPelatihan = row['Kelas Pelatihan']
        PelatihanHariKe = row['Pelatihan Hari Ke-']
        TandaTanganJelas = convertToBinaryData(row['Tanda Tangan Jelas'])
        tuple_data = (NIM, NamaLengkap, ProgramStudi, KelasPelatihan, PelatihanHariKe, TandaTanganJelas)
        counting = insertToMySQL(tuple_data)
        count = count + counting

print("{} Data and Tanda Tangan have been inserted successfully as a OLE into UINSA".format(count))

Found File processing :hasil-0.csv
Found File processing :hasil-1.csv
Found File processing :hasil-2.csv
Found File processing :hasil-3.csv
Found File processing :hasil-4.csv
Processing Upload for :hasil-0.csv
Processing Upload for :hasil-1.csv
Processing Upload for :hasil-2.csv
Processing Upload for :hasil-3.csv
Failed inserting BLOB data into MySQL table 1054 (42S22): Unknown column 'nan' in 'field list'
Processing Upload for :hasil-4.csv
13511 Data and Tanda Tangan have been inserted successfully as a OLE into UINSA


In [ ]:
# Percobaan Import Ke MySQL

def convertToBinaryData(filename):
    # Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData

def insertToMySQL(dataOneRow):
    try:
        connection = sql.connect(host='localhost', 
                                database='mahasiswa', 
                                user='lanang_afkaar', 
                            password='1q2w3e4r5t')
        cursor = connection.cursor()
        sql_insert = """INSERT INTO UINSA
                        (NIM, NamaLengkap, ProgramStudi, 
                        KelasPelatihan, PelatihanHariKe, TandaTanganJelas)
                        VALUES (%s, %s, %s, %s, %s, %s)"""
        result  = cursor.execute(sql_insert, dataOneRow)
        connection.commit()
        return 1
    
    except sql.Error as error:
        print("Failed inserting BLOB data into MySQL table {}".format(error))
        return 0

# Convert data into tuple format

df_mahasiswa = pd.read_csv('hasil.csv')
count = 0
for index, row in df_mahasiswa.iterrows():
    NIM = row['NIM']
    NamaLengkap = row['Nama Lengkap']
    ProgramStudi = row['Program Studi']
    KelasPelatihan = row['Kelas Pelatihan']
    PelatihanHariKe = row['Pelatihan Hari Ke-']
    TandaTanganJelas = convertToBinaryData(row['Tanda Tangan Jelas'])
    tuple_data = (NIM, NamaLengkap, ProgramStudi, KelasPelatihan, PelatihanHariKe, TandaTanganJelas)
    counting = insertToMySQL(tuple_data)
    count = count + counting

print("{} Data and Tanda Tangan have been inserted successfully as a OLE into UINSA".format(count))
   

In [ ]:
import pyodbc

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=D:\Pyshit\frompython.accdb;')
cursor = conn.cursor()
sql_query = 'select * from DaftarKehadiran'
cursor.execute(sql_query)
records = cursor.fetchall()
for row in records:
    print (row)

In [ ]:
import pyodbc

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=D:\Pyshit\frompython.accdb;')
cursor = conn.cursor()
sql_query = 'select * from DaftarKehadiran'
cursor.execute(sql_query)
records = cursor.fetchall()
for row in records:
    print (row)

In [43]:
df_mahasiswa = pd.DataFrame(list(records))
df_mahasiswa

,0
0,"[B94217101, Isaatun Adillia Rasyid, Manajemen ..."
1,"[B94217101, Isaatun Adillia Rasyid, Manajemen ..."


In [ ]:
import mysql.connector as sql

connection = sql.connect(host='localhost', 
                                        database='integrated_data', 
                                        user='lanang_afkaar', 
                                        password='1q2w3e4r5t')
sql_select_Query = "SELECT * FROM {}".format("salary_and_employee_perfomances_2020")
cursor = connection.cursor()
cursor.execute(sql_select_Query)
records = cursor.fetchall()
num_records = cursor.rowcount
print(records)
print("Total number of rows in table: ", num_records)

In [ ]:
import pyodbc

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=D:\Pyshit\frompython.accdb;')
cursor = conn.cursor()
sql_query = 'select * from DaftarKehadiran'
cursor.execute(sql_query)
records = cursor.fetchall()
for row in records:
    print (row)

In [ ]:
# Percobaan Export ke MS Access

import pyodbc
import pandas as pd
import PIL

def convertToBinaryData(filename):
    # Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\Users\afkaa\Google Drive\Multimatics\Data\UINSA\UINSA_afkaar.accdb;')
cursor = conn.cursor()

# Convert data into tuple format
NIM = "B94217101"
NamaLengkap = 'Isaatun Adillia Rasyid'
ProgramStudi = "Manajemen Dakwah"
TandaTanganJelas = convertToBinaryData("img_out/X6.png")
print(TandaTanganJelas)
tuple_data = (NIM, NamaLengkap, ProgramStudi, KelasPelatihan, PelatihanHariKe, TandaTanganJelas)

sql_insert = """INSERT INTO tbPeserta
                (NIM, NamaLengkap, ProgramStudi, TdTgn)
                VALUES (%s, %s, %s, %s)"""
result  = cursor.execute(sql_insert, tuple_data)
connection.commit()

print("Image and file inserted successfully as a OLE into DaftarMahasiswa")